In [193]:
from bs4 import BeautifulSoup
import requests
import math
import pandas as pd
import os.path
import sys


file_name = 'VintageWatches.csv'

def getNumberOfwatches(textContent):
    numberWatchesText=textContent[2] 
    r=numberWatchesText.find(' results')
    l=numberWatchesText.find('of')
    totalNumberOfWatches=int(numberWatchesText[l+3:r])
    numberWatchesText=numberWatchesText[9:l]
    r=numberWatchesText.find('–')
    LastnumberWatchesThisPage=int(numberWatchesText[r+1:-1])
    FirstnumberWatchesThisPage=int(numberWatchesText[0:r])
    print('total number of watches in webshop {}'.format(totalNumberOfWatches))
    WonPage =LastnumberWatchesThisPage-FirstnumberWatchesThisPage+1
    print('{} watches on this page'.format(WonPage))
    print('total number of pages {}'.format(math.ceil(int(totalNumberOfWatches)/int(LastnumberWatchesThisPage))))
                        
    return totalNumberOfWatches,FirstnumberWatchesThisPage,LastnumberWatchesThisPage,math.ceil(int(totalNumberOfWatches)/int(LastnumberWatchesThisPage))

def unescape(s):
    s = s.replace("&lt;", "<")
    s = s.replace("&gt;", ">")
    # this has to be last:
    s = s.replace("&amp;", "&")
    return s

#p=s.split('name product-title')
def haalWatchesVanPagina(p):
    w=[]
    for nr in range(1,len(p)):
        q=p[nr][50:-1]
        q1=q.find('\n')+5
        q2=q[q1:-1]
        q1=q2.find('\n')
        item  = q2[:q1]
        Watch = {'Item':item.strip()}
        r=q2[q2.find('€')+14:-1].find('<')
        q3=q2[q2.find('€')+14:q2.find('€')+12+r]
        q3=q3[q3.find(' '):-5]
        P=q3.strip()
        if '.' not in P:
            P='Not available'
        keep = ''
        aa=q.split('https')
        for a in aa:
            if '.jpg' in a:
                if '-front-' in a.lower():
                    r=a.find('.jpg')
                    a='https'+a[0:r+4]
                    if '1024' in a:
                        keep = a
        if len(keep)>0:
            a = keep
        Watch = {'Item':item.strip(),'Price':P,'URL':a}
        w.append(Watch.copy())
    return w

def findImage(SP):
    r=''
    s=SP.prettify()
    I=s.split('.jpg')
    for i in I:
        s=i[i.find('https'):-1]
        if len(s)>0:
            r=s+'.jpg'
    return r

def getPageStuff(page_link):
    page_response = requests.get(page_link, timeout=5)
    page_content = BeautifulSoup(page_response.content, "html.parser")
    textContent = []
    i=0
    while True:
        paragraphs = page_content.find_all("p")[i].text
        textContent.append(paragraphs)
        i+=1
        if  paragraphs == 'ABOUT':
            break
    return textContent,page_content

def getFirstPass(html_soup):
    w=[]
    d=[]
    sp=html_soup.find_all('img')
    for s in sp[2:50]:    
        if 'srcset="https://amsterdamvintagewatches.com/wp-content/uploads/' in s.prettify():
            x=s.prettify()
            if 'fro' in x.lower():
                s = findImage(s)
                w.append(s)
                sl=s.find('uploads')
                a=s[sl+8:sl+40]
                r=a.rfind('A')
                d.append(a[:r-1])

    return d,w





def getInfoOnWatch(w):
    index = next((index for (index, d) in enumerate(items) if d["Item"] == w), None)
    return index


if os.path.isfile(file_name):
    MyWatches = pd.read_csv(file_name,sep='\t', encoding='utf-8')
else:
    MyWatches=pd.DataFrame(columns=['watch', 'img_link', 'price','date'])

page_link = 'https://amsterdamvintagewatches.com/shop'



textContent = []
textContent,page_content =getPageStuff(page_link)
a,b,c,NrPages =getNumberOfwatches(textContent)
print('pagina 1')
print(NrPages)
for i in range(1,NrPages+1):
    tempDf = tempDf.iloc[0:0]
    print('reading page {} of {}'.format(i,NrPages))
    textContent = []
    page_link = 'https://amsterdamvintagewatches.com/shop'
    if i>1:
        page_link = page_link+'/page/'+str(i)+'/'
    textContent,page_content =getPageStuff(page_link)
    d,w =getFirstPass(page_content)
    a1 = page_content.find(class_='shop-container')
    try:
        a2=a1.find_all('div')
        s=a2[1].prettify()
        items = haalWatchesVanPagina(s.split('name product-title'))
    
    finally:
        pass
    for i in items:
        klok = i.get('Item')
        price = i.get('Price')
        url = i.get('URL')
        
        
        s = price
        s = s.replace(".", "")
        try:
            price=int(s)
            price=str(s)
        except:
            price='Not available'
        finally:
            pass
        if 'https' not in url:
            date='not avialable'
            url='error in retrieving'
        else:
            d1=url.split('/uploads/')[1]
            date=d1[0:d1.rfind('/')]
        MyWatches = MyWatches.append({'watch' : klok , 'price' : price,'img_link':url,'date':date} , ignore_index=True)

MyWatches.drop_duplicates(keep=False)  
MyWatches = MyWatches[['watch', 'img_link', 'price', 'date']]
mx=MyWatches[MyWatches.price!='Not available'].price.astype(int).max()
mi=MyWatches[MyWatches.price!='Not available'].price.astype(int).min()
print('----------------')
print('Most expensive item: {}'.format(mx))
print('Most affordable watch: {}'.format(mi))




print('Saving database')
MyWatches.to_csv(file_name, sep='\t', encoding='utf-8')
MyWatches



total number of watches in webshop 144
24 watches on this page
total number of pages 6
pagina 1
6
reading page 1 of 6
reading page 2 of 6
reading page 3 of 6
reading page 4 of 6
reading page 5 of 6
reading page 6 of 6
----------------
Most expensive item: 65000
Most affordable watch: 1265
Saving database


,watch,img_link,price,date
0,Patek Philippe Aquanaut 5167A-001 Full Set,https://amsterdamvintagewatches.com/wp-content...,Not available,2019/01
1,Omega Constellation 1438110 ‘Pie-Pan’,https://amsterdamvintagewatches.com/wp-content...,Not available,2018/11
2,Rolex Submariner 5513,https://amsterdamvintagewatches.com/wp-content...,8900,2019/02
3,Patek Philippe Calatrava 565 ‘Breguet Numerals’,https://amsterdamvintagewatches.com/wp-content...,39000,2018/10
4,Patek Philippe Calatrava 2511 ‘Oversized’,https://amsterdamvintagewatches.com/wp-content...,12600,2019/02
5,Rolex Day-Date 18238 ‘MOP’,https://amsterdamvintagewatches.com/wp-content...,14900,2019/01
6,Rolex Air-King 14000M,https://amsterdamvintagewatches.com/wp-content...,Not available,2018/12
7,Rolex Air-King 14000,https://amsterdamvintagewatches.com/wp-content...,Not available,2019/02
8,Rolex Air-King 14000,https://amsterdamvintagewatches.com/wp-content...,Not available,2018/02
9,Rolex Datejust 16263 ‘Turn-O-Graph’,https://amsterdamvintagewatches.com/wp-content...,4500,2019/01
